In [1]:
import numpy as np
import pandas as pd
import datasets
from seacrowd import SEACrowdConfigHelper, Tasks

from itertools import chain
import json
from collections import Counter

In [2]:
dset_df = pd.read_csv('dataset_list.csv')

In [3]:
sc_conhelp = SEACrowdConfigHelper()

 74%|██████████████████████████████████████████████████████████████████████████████████████████████████████▉                                     | 250/340 [00:03<00:01, 70.63it/s]/home/samuel/anaconda2/envs/env_embedding/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for gem contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/gem
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 340/340 [00:35<00:00,  9.68it/s]


# SEACrowd Dataset

In [4]:
def retrieve_text(dset, config_name):
    if 'paracotta_id' in config_name:
        return dset['train']['text_1'] + dset['train']['text_2']

    if 'aya_dataset' in config_name:
        return dset['train']['text_1'] + dset['train']['text_2']

    if 'codeswitch_reddit' in config_name:
        return dset['train']['text']

    if 'mkqa' in config_name:
        return dset['train']['question']

    if 'yunshan_cup_2020' in config_name:
        return list(map(lambda x: ' '.join(x['tokens']), dset['train'])) + list(map(lambda x: ' '.join(x['tokens']), dset['validation']))

    if 'mozilla_pontoon' in config_name:
        return dset['train']['text_2']

    if 'ntrex_128' in config_name:
        return dset['test']['text_1']

    if 'nusax_mt' in config_name:
        return dset['train']['text_1']

    if 'tatoeba' in config_name:
        return dset['validation']['text_1']

    if 'xcopa' in config_name:
        return dset['validation']['context'] + dset['validation']['context'] + list(chain.from_iterable(list(dset['validation']['choices'])))

    if 'bactrian_x' in config_name:
        return dset['train']['text_1'] + dset['train']['text_2']

    if 'ara_close' in config_name:
        return dset['train']['text']

    if 'cosem' in config_name:
        return dset['train']['text']

    if 'nusaparagraph_rhetoric' in config_name:
        return dset['test']['text']

    if 'emotes_3k' in config_name and 'seacrowd_text' in config_name:
        return dset['test']['text']

    if 'emotes_3k' in config_name and 'seacrowd_t2t' in config_name:
        return dset['test']['text_1'] + dset['test']['text_2']

    if 'sea_bench' in config_name:
        return dset['train']['text_1'] + dset['train']['text_2']

    if 'seaeval_cross_mmlu' in config_name:
        return dset['test']['question'] + list(map(lambda x: x[0], dset['test']['answer']))

    if 'seaeval_cross_logiqa' in config_name:
        return dset['test']['question'] + list(map(lambda x: x[0], dset['test']['answer'])) + list(chain.from_iterable(list(dset['test']['choices'])))

    if 'belebele' in config_name:
        return dset['test']['question'] + dset['test']['context'] + list(chain.from_iterable(list(dset['test']['choices'])))

    if 'khmer_alt_pos' in config_name:
        return list(map(lambda x: ' '.join(x['tokens']), dset['train']))

    if 'korpus_nusantara' in config_name:
        return dset['train']['text_1']

    if 'nusatranslation_mt' in config_name:
        return dset['test']['text_1']

    if 'parallel_asian_treebank' in config_name:
        return dset['test']['text_1']

    raise ValueError(f'Unknown Config Name `{config_name}`')

In [5]:
valid_config_names = {}
for split in ['Train', 'Test']:
    valid_config_names[split] = {}
    for label_type in ['Machine-translated', 'Natural', 'Human-translated']:
        valid_config_names[split][label_type] = []
        sc_dset_names = dset_df.loc[(dset_df['split'] == split) & (dset_df['type'] == label_type), 'dataloader_name'].tolist()
        for con in sc_conhelp.filtered(
            lambda x: (x.dataset_name in sc_dset_names) and ('source' not in x.config.name)
        ):
            if 'cc3m_35l' in con.dataset_name:
                continue

            lang_list = dset_df.loc[dset_df['dataloader_name'] == con.dataset_name, 'lang'].values[0].split(',')
            if len(lang_list) > 1:
                if 'bactrian_x' in con.dataset_name:
                    # Bactrian config name use ISO639-2 instead of ISO639-3, so just force it in
                    valid_config_names[split][label_type].append(con.config.name)
                else:
                    for lang in lang_list:
                        if Tasks.MACHINE_TRANSLATION in list(con.tasks):
                            # Skip Redundant MT Config
                            if ('ntrex' in con.config.name and 'eng-US_seacrowd' not in con.config.name) or \
                                ('nusax_mt' in con.config.name and 'ind_seacrowd' not in con.config.name) or \
                                ('parallel_asian_treebank' in con.config.name and 'eng_seacrowd' not in con.config.name) or \
                                ('korpus_seacrowd' in con.config.name and 'ind_seacrowd' not in con.config.name) or \
                                ('nusatranslation_mt_' in con.config.name and 'ind_seacrowd' not in con.config.name):
                                continue
                            if lang in con.config.name:
                                valid_config_names[split][label_type].append(con.config.name)
                        else:
                            if lang in con.config.name:
                                valid_config_names[split][label_type].append(con.config.name)
            else:
                if 'emotes' in con.config.name:
                    if lang_list[0] in con.config.name:
                        valid_config_names[split][label_type].append(con.config.name)
                elif Tasks.MACHINE_TRANSLATION in list(con.tasks):
                    # Skip Redundant MT Config
                    if ('ntrex' in con.config.name and 'eng-US_seacrowd' not in con.config.name) or \
                        ('nusax_mt' in con.config.name and 'ind_seacrowd' not in con.config.name) or \
                        ('parallel_asian_treebank' in con.config.name and 'eng_seacrowd' not in con.config.name) or \
                        ('korpus_seacrowd' in con.config.name and 'ind_seacrowd' not in con.config.name):
                        continue
                    valid_config_names[split][label_type].append(con.config.name)
                else:
                    valid_config_names[split][label_type].append(con.config.name)

In [6]:
valid_config_names

{'Train': {'Machine-translated': ['paracotta_id_seacrowd_t2t'],
  'Natural': ['aya_dataset_ceb_seacrowd_t2t',
   'aya_dataset_ind_seacrowd_t2t',
   'aya_dataset_jav_seacrowd_t2t',
   'aya_dataset_mya_seacrowd_t2t',
   'aya_dataset_sun_seacrowd_t2t',
   'aya_dataset_tha_seacrowd_t2t',
   'aya_dataset_vie_seacrowd_t2t',
   'aya_dataset_zsm_seacrowd_t2t',
   'codeswitch_reddit_eng_monolingual_seacrowd_ssp',
   'mkqa_khm_seacrowd_qa',
   'mkqa_zsm_seacrowd_qa',
   'mkqa_tha_seacrowd_qa',
   'mkqa_vie_seacrowd_qa',
   'yunshan_cup_2020_seacrowd_seq_label'],
  'Human-translated': ['mozilla_pontoon_eng_mya_seacrowd_t2t',
   'mozilla_pontoon_eng_ceb_seacrowd_t2t',
   'mozilla_pontoon_eng_ind_seacrowd_t2t',
   'mozilla_pontoon_eng_jav_seacrowd_t2t',
   'mozilla_pontoon_eng_khm_seacrowd_t2t',
   'mozilla_pontoon_eng_lao_seacrowd_t2t',
   'mozilla_pontoon_eng_tgl_seacrowd_t2t',
   'mozilla_pontoon_eng_tha_seacrowd_t2t',
   'mozilla_pontoon_eng_vie_seacrowd_t2t',
   'ntrex_128_ind_eng-US_seacrowd_

In [7]:
%%time
config_stats = {}
train_data = {'text': [], 'label': [], 'config': []}
test_data = {'text': [], 'label': [], 'config': []}

data = {'Train': train_data, 'Test': test_data}
for split in valid_config_names.keys():
    for label_type in valid_config_names[split].keys():
        for config_name in valid_config_names[split][label_type]:
            texts = retrieve_text(sc_conhelp.for_config_name(config_name).load_dataset(), config_name)
            labels = [label_type for _ in range(len(texts))]
            configs = [config_name for _ in range(len(texts))]
            
            config_stats[config_name] = len(texts)
            data[split]['text'] += texts
            data[split]['label'] += labels
            data[split]['config'] += configs
config_stats

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Index(['id', 'ind_text', 'tgt_text'], dtype='object')


...

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

CPU times: user 1min 11s, sys: 2.29 s, total: 1min 13s
Wall time: 1min 12s


{'paracotta_id_seacrowd_t2t': 12000000,
 'aya_dataset_ceb_seacrowd_t2t': 1454,
 'aya_dataset_ind_seacrowd_t2t': 1572,
 'aya_dataset_jav_seacrowd_t2t': 494,
 'aya_dataset_mya_seacrowd_t2t': 944,
 'aya_dataset_sun_seacrowd_t2t': 388,
 'aya_dataset_tha_seacrowd_t2t': 1448,
 'aya_dataset_vie_seacrowd_t2t': 17352,
 'aya_dataset_zsm_seacrowd_t2t': 20146,
 'codeswitch_reddit_eng_monolingual_seacrowd_ssp': 190250,
 'mkqa_khm_seacrowd_qa': 10000,
 'mkqa_zsm_seacrowd_qa': 10000,
 'mkqa_tha_seacrowd_qa': 10000,
 'mkqa_vie_seacrowd_qa': 10000,
 'yunshan_cup_2020_seacrowd_seq_label': 8000,
 'mozilla_pontoon_eng_mya_seacrowd_t2t': 13699,
 'mozilla_pontoon_eng_ceb_seacrowd_t2t': 470,
 'mozilla_pontoon_eng_ind_seacrowd_t2t': 43789,
 'mozilla_pontoon_eng_jav_seacrowd_t2t': 685,
 'mozilla_pontoon_eng_khm_seacrowd_t2t': 14481,
 'mozilla_pontoon_eng_lao_seacrowd_t2t': 15302,
 'mozilla_pontoon_eng_tgl_seacrowd_t2t': 12252,
 'mozilla_pontoon_eng_tha_seacrowd_t2t': 37691,
 'mozilla_pontoon_eng_vie_seacrowd_t

# Custom Data 
- Aya Collection
- Aya Evaluation Suite
- CC3M-35L

In [8]:
aya_col_dset = datasets.load_dataset('CohereForAI/aya_collection', name='translated_dolly')
aya_col_dset = aya_col_dset.filter(lambda x: x['language'] in ['ceb','tha','mya','zsm','jav','ind','vie','sun','khm','lao','eng'])
texts = aya_col_dset['train']['inputs'] + aya_col_dset['train']['targets']
labels = ['Machine-translated' for _ in range(len(texts))]
configs = list(map(lambda lang: 'aya_collection_'+lang+'_seacrowd_t2t', aya_col_dset['train']['language'])) * 2

train_data['text'] += texts
train_data['label'] += labels
train_data['config'] += configs

In [9]:
aya_eval_dset = datasets.load_dataset('CohereForAI/aya_evaluation_suite', name='dolly_machine_translated')
aya_eval_dset = aya_eval_dset.filter(lambda x: x['language'] in ['ceb','tha','mya','zsm','jav','ind','vie','sun','khm','lao','eng'])
texts = aya_eval_dset['test']['inputs'] + aya_eval_dset['test']['targets']
labels = ['Machine-translated' for _ in range(len(texts))]
configs = list(map(lambda lang: 'aya_evaluation_suite_'+lang+'_seacrowd_t2t', aya_eval_dset['test']['language'])) * 2

test_data['text'] += texts
test_data['label'] += labels
test_data['config'] += configs

In [10]:
%%time
texts, labels, configs = [], [], []
lang_map = {'id': 'ind', 'th': 'tha', 'fil': 'tgl', 'vi': 'vie'}
with open('./cc3m_mt_train.jsonl') as f:
    for line in f:
        row = json.loads(line)
        if row['trg_lang'] in ['fil', 'tgl', 'ind', 'tha', 'vie', 'tl','id','th','vi']:
            texts.append(row['translation_tokenized'])
            labels.append('Machine-translated')
            configs.append('cc3m_mt_' + lang_map[row['trg_lang']] + '_seacrowd_imtext')

train_data['text'] += texts
train_data['label'] += labels
train_data['config'] += configs

CPU times: user 9min 15s, sys: 16 s, total: 9min 31s
Wall time: 9min 30s


In [11]:
# Sanity Check
print('Train: ', len(train_data['text']), len(train_data['label']), len(train_data['config']))
print('Test: ', len(test_data['text']), len(test_data['label']), len(test_data['config']))

Train:  26081150 26081150 26081150
Test:  2347026 2347026 2347026


In [12]:
import datasets
train_dset = datasets.Dataset.from_dict(train_data)
test_dset = datasets.Dataset.from_dict(test_data)

In [14]:
datasets.DatasetDict({
    'train': train_dset,
    'test': test_dset
}).push_to_hub("SEACrowd/sea_translationese")

Uploading the dataset shards:   0%|          | 0/8 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3261 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3261 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3261 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3261 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/3261 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1174 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1174 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/SEACrowd/sea_translationese/commit/e45353d000385c2621edcfd44a0c993314d0b11c', commit_message='Upload dataset', commit_description='', oid='e45353d000385c2621edcfd44a0c993314d0b11c', pr_url=None, pr_revision=None, pr_num=None)

In [15]:
train_df = train_dset.to_pandas()
test_df = test_dset.to_pandas()

In [16]:
train_df.groupby('label').size()

label
Human-translated        199846
Machine-translated    25599256
Natural                 282048
dtype: int64

In [17]:
train_df.groupby('config').size()

config
aya_collection_ceb_seacrowd_t2t                      29616
aya_collection_eng_seacrowd_t2t                      29616
aya_collection_ind_seacrowd_t2t                      29616
aya_collection_jav_seacrowd_t2t                      29616
aya_collection_khm_seacrowd_t2t                      29616
aya_collection_lao_seacrowd_t2t                      29616
aya_collection_mya_seacrowd_t2t                      29616
aya_collection_sun_seacrowd_t2t                      29616
aya_collection_tha_seacrowd_t2t                      29616
aya_collection_vie_seacrowd_t2t                      29616
aya_collection_zsm_seacrowd_t2t                      29616
aya_dataset_ceb_seacrowd_t2t                          1454
aya_dataset_ind_seacrowd_t2t                          1572
aya_dataset_jav_seacrowd_t2t                           494
aya_dataset_mya_seacrowd_t2t                           944
aya_dataset_sun_seacrowd_t2t                           388
aya_dataset_tha_seacrowd_t2t                     

In [18]:
test_df.groupby('label').size()

label
Human-translated       120319
Machine-translated     808604
Natural               1418103
dtype: int64

In [22]:
test_df.groupby('config').size()

config
ara_close_bcl_seacrowd_text                         150
ara_close_ceb_seacrowd_text                         349
aya_evaluation_suite_ceb_seacrowd_t2t               400
aya_evaluation_suite_eng_seacrowd_t2t               400
aya_evaluation_suite_ind_seacrowd_t2t               400
aya_evaluation_suite_jav_seacrowd_t2t               400
aya_evaluation_suite_khm_seacrowd_t2t               400
aya_evaluation_suite_lao_seacrowd_t2t               400
aya_evaluation_suite_mya_seacrowd_t2t               400
aya_evaluation_suite_sun_seacrowd_t2t               400
aya_evaluation_suite_tha_seacrowd_t2t               400
aya_evaluation_suite_vie_seacrowd_t2t               400
aya_evaluation_suite_zsm_seacrowd_t2t               400
bactrian_x_id_seacrowd_t2t                       134034
bactrian_x_km_seacrowd_t2t                       134034
bactrian_x_my_seacrowd_t2t                       134034
bactrian_x_th_seacrowd_t2t                       134034
bactrian_x_tl_seacrowd_t2t               

# Sandbox

In [ ]:
dset = sc_conhelp.for_config_name('paracotta_id_seacrowd_t2t').load_dataset()
# dset['train']['text_1'] + dset['train']['text_2']

In [ ]:
dset = sc_conhelp.for_config_name('aya_dataset_ceb_seacrowd_t2t').load_dataset()
# dset['train']['text_1'] + dset['train']['text_2']

In [219]:
dset = sc_conhelp.for_config_name('codeswitch_reddit_eng_monolingual_seacrowd_ssp').load_dataset()
# dset['train']['text']

In [220]:
dset = sc_conhelp.for_config_name('mkqa_khm_seacrowd_qa').load_dataset()
# dset['train']['question']

In [221]:
dset = sc_conhelp.for_config_name('yunshan_cup_2020_seacrowd_seq_label').load_dataset()
# list(map(lambda x: ' '.join(x['tokens']), dset['train'])) + list(map(lambda x: ' '.join(x['tokens']), dset['validation']))

In [222]:
dset = sc_conhelp.for_config_name('mozilla_pontoon_eng_lao_seacrowd_t2t').load_dataset()
# dset['train']['text_2']

In [223]:
dset = sc_conhelp.for_config_name('ntrex_128_lao_eng-US_seacrowd_t2t').load_dataset()
# dset['train']['text_1']

In [224]:
dset = sc_conhelp.for_config_name('nusax_mt_jav_ind_seacrowd_t2t').load_dataset()
# dset['train']['text_1']

In [225]:
dset = sc_conhelp.for_config_name('tatoeba_ind_seacrowd_t2t').load_dataset()
# dset['validation']['text_1']

In [226]:
dset = sc_conhelp.for_config_name('xcopa_vie_seacrowd_qa').load_dataset()
# dset['validation']['context'] + dset['validation']['context'] + list(chain.from_iterable(list(dset['validation']['choices'])))

In [227]:
dset = sc_conhelp.for_config_name('bactrian_x_my_seacrowd_t2t').load_dataset()
# dset['train']['text_1'] + dset['train']['text_2']

In [228]:
dset = sc_conhelp.for_config_name('ara_close_bcl_seacrowd_text').load_dataset()
# dset['train']['text']

In [229]:
dset = sc_conhelp.for_config_name('cosem_seacrowd_ssp').load_dataset()
# dset['train']['text']

In [230]:
dset = sc_conhelp.for_config_name('nusaparagraph_rhetoric_jav_seacrowd_text').load_dataset()
# dset['test']['text']

In [231]:
dset = sc_conhelp.for_config_name('emotes_3k_eng_seacrowd_text').load_dataset()
# dset['test']['text']

In [232]:
dset = sc_conhelp.for_config_name('emotes_3k_eng_seacrowd_t2t').load_dataset()
# dset['test']['text_1'] + dset['test']['text_2']

In [233]:
dset = sc_conhelp.for_config_name('sea_bench_tgl_seacrowd_t2t').load_dataset()
# dset['train']['text_1'] + dset['train']['text_2']

In [235]:
dset = sc_conhelp.for_config_name('seaeval_cross_mmlu_ind_seacrowd_qa').load_dataset()
# dset['test']['question'] + list(map(lambda x: x[0], dset['test']['answer']))

In [236]:
dset = sc_conhelp.for_config_name('seaeval_cross_logiqa_ind_seacrowd_qa').load_dataset()
# dset['test']['question'] + list(map(lambda x: x[0], dset['test']['answer'])) + list(chain.from_iterable(list(dset['test']['choices'])))

In [237]:
dset = sc_conhelp.for_config_name('belebele_ceb_latn_seacrowd_qa').load_dataset()
# dset['test']['question'] + dset['test']['context'] + list(chain.from_iterable(list(dset['test']['choices'])))

In [239]:
dset = sc_conhelp.for_config_name('khmer_alt_pos_seacrowd_seq_label').load_dataset()
# list(map(lambda x: ' '.join(x['tokens']), dset['train']))

In [239]:
dset = sc_conhelp.for_config_name('korpus_seacrowd_jav_ind_seacrowd_t2t').load_dataset()
# dset['test']['text_1']

In [247]:
dset = sc_conhelp.for_config_name('nusatranslation_mt_jav_ind_seacrowd_t2t').load_dataset()
# dset['test']['text_1']

In [250]:
dset = sc_conhelp.for_config_name('parallel_asian_treebank_khm_eng_seacrowd_t2t').load_dataset()
# dset['test']['text_1']

In [252]:
dset['train'][:3]

{'id': ['0', '1', '2'],
 'text_1': ['លោកស្ទេវ រ៉ាយថ៍ ដែលកាលពីម្សិលមិញត្រូវបានផ្តន្ទាទោសលើឃាតកម្មលើស្ត្រីប្រាំនាក់នៅអ៊ីបស្វីជ រដ្ឋសឺហ្វហ្វក ប្រទេសអង់គ្លេស នៅថ្ងៃនេះត្រូវបានកាត់ទោសនៅតុលាការក្រោនអ៊ីបស្វីជ ឱ្យជាប់ពន្ធនាគារអស់មួយជីវិត។ សាកសពនៃស្ត្រីប្រាំនាក់ដែលបានធ្វើការជាអ្នករកស៊ីផ្លូវភេទនៅក្នុង អ៊ីបស្វីជ ត្រូវបានរកឃើញនៅជុំវិញទីក្រុងនេះ ក្នុងខែធ្នូ ឆ្នាំ2006។ ចៅក្រម លោកចាស់ស្ទីក ហ្គ្រស់ បាននិយាយថា " កម្រិតធ្ងន់នៃការគិតទុកជាមុននិងការគ្រោងទុក" គឺវាទាក់ទងនឹងអត្ថន័យដែលចាំបាច់សំរាប់ការកាត់ទោសអស់មួយជីវិតដល់លោករ៉ាយថ៍ ត្រូវបានសម្រេច។ "វាត្រឹមត្រូវហើយ ដែលអ្នកគួរតែចំណាយជីវិតទាំងមូលរបស់អ្នកនៅក្នុងពន្ធនាគារ។" ថ្លែងបន្ទាប់ពីការកាត់ទោសនេះ អនុប្រធានតម្រួត នៃប៉ូលីស រដ្ឋសឺហ្វហ្វក លោកស្រី ចាកឃ្វី ឈា បាននិយាយថា "នៅពេលចាប់ផ្តើមការស៊ើបអង្កេតយើងមិនអាចស្នើសុំអ្វីបន្ថែមទៀតបានទេ។" "វាគឺជាគុណសម្បត្តិរបស់អ្នកទាំងអស់ដែលចូលរួមពាក់ព័ន្ធ - មិនត្រឹមតែមន្រ្តីប៉ូលីសប៉ុណ្ណោះទេ ប៉ុន្តែរួមទាំងក្រុមគាំទ្ររបស់ពួកគេ និងសមាជិកសាធារណៈទាំងអស់ដែលបានទូរស័ព្ទមក ក្នុងការផ្តល់ជូនព័ត៌មាន។"',
  'សារព័ត៌មាន Wikinews មានបទសម្ភាសន៍ដោយឡែកជាមួ

In [261]:
dset_df.loc[(~dset_df['split'].isna()), 'dataloader_name'].tolist()

27

In [12]:
%%time
c = Counter()
with open('./cc3m_mt_train.jsonl') as f:
    for line in f:
        row = json.loads(line)
        if row['trg_lang'] in ['fil', 'tgl', 'ind', 'tha', 'vie', 'tl','id','th','vi']:
            c[row['trg_lang']] += 1

CPU times: user 9min 9s, sys: 13.9 s, total: 9min 23s
Wall time: 9min 23s


In [13]:
c

Counter({'id': 3318370, 'th': 3318370, 'fil': 3318370, 'vi': 3318370})